# Beispiel für Experiment Tracking

Ein lokaler MLFlow Tracking Server wird aus der Konsole mit folgendem Befehl gestartet:

In [ ]:
!mlflow ui

Eine GUI ist nun unter http://localhost:5000 erreichbar.

## Tracking eines Experiments für Cracktip Detection Modelle

Import benötigter Module

In [53]:
import mlflow
import example.data_loader as dl
import example.model_loader as ml
import example.train_utils as tu
import torch
from torch.utils.data import Subset
from torch.utils.data.dataloader import DataLoader

In [54]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Laden der Daten (Es werden zu demonstrationszwecken nur 50 Beispiele geladen)

In [55]:
train_set, val_set = dl.load_data()
train_set = Subset(train_set, range(50))
val_set = Subset(val_set, range(50))

### Configuration eines Trainingsdurchlaufs
Hier werden zentral die Parameter für das Training und die Konfiguration des Modells festgelegt.
Für verschiedene Durchläufe ist dies der einzige Punkt an dem Einstellungen vorgenommen werden müssen.

In [67]:
model_params = {
    'model_size': 64,
    'model_dropout_prob': 0.1
}

train_params = {
    'train_epochs': 3,
    'train_learning_rate': 1e-4,
    'train_batch_size': 5,
    'optimizer': 'Adam'
}

### Durchführen eines Trainingsdurchlaufs mit Tracking über MLFlow

Alle Parameter, die zur **Reproduktion** des Durchlaufs wichtig sind, werden dokumentiert.

Der Traininigsfortschritt wird laufend überwacht. Anpassung des Modells über die *Loss* Funktion und Güte des Modells nach jeder Trainingsepoche.

Zunächst wird dafür eine Verbindung zum MLFlow Tracking Server hergestellt. Danach wird der Durchlauf mit dem Experiment *crack_tip_detection* verbunden.

In [68]:
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('/crack_tip_detection')

<Experiment: artifact_location='mlflow-artifacts:/452056258848977975', creation_time=1710687793008, experiment_id='452056258848977975', last_update_time=1710687793008, lifecycle_stage='active', name='/crack_tip_detection', tags={}>

In [69]:
with mlflow.start_run():
    mlflow.set_tag('Training Info', 'Cracktip detection tests') # Durchläufe können mit Tags versehen werden.
    mlflow.log_params(model_params) # Logging der Modellparameter
    mlflow.log_params(train_params) # Logging der Trainingsparameter

    # Instanstiieren des Modells und der Daten anhand der zuvor festgelegten Parameter.
    model = ml.load_model(size=model_params['model_size'], 
                          dropout_prob=model_params['model_dropout_prob'])
    train_loader = DataLoader(train_set, batch_size=train_params['train_batch_size'])
    val_loader = DataLoader(val_set, batch_size=train_params['train_batch_size'])
    optimizer = torch.optim.Adam(model.parameters(), lr=train_params['train_learning_rate'], 
                           amsgrad=True)

    # Trainingsschleife
    for epoch in range(train_params['train_epochs']):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels[0].to(device), labels[1].to(device)
            # zero the gradients
            optimizer.zero_grad()

            outputs = model.forward(inputs)
            loss = tu.compute_loss(outputs, labels)
            
            if i % 2 == 0:
                mlflow.log_metric('loss', loss)
                
            loss.backward()
            optimizer.step()

        # Validieren des Modells anhand von Testdaten
        deviation, reliability = tu.validate_model(model, val_loader)
        mlflow.log_metric('deviation', deviation) # Logging der Abeweichung von den Ground Truth Daten
        mlflow.log_metric('reliability', reliability) # Logging der Berechneten zuverlässigkeit

    # Logging und Registrierung des Trainierten Modells
    mlflow.pytorch.log_model(
        pytorch_model=model,
        artifact_path='crack-tip-detection',
        registered_model_name='crack-tip-detection-production',
    )

Registered model 'crack-tip-detection-production' already exists. Creating a new version of this model...
2024/03/17 19:31:11 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: crack-tip-detection-production, version 2
Created version '2' of model 'crack-tip-detection-production'.


In [71]:
mlflow.end_run()

Die Dokumentation des Durchlaufs kann nun über den Trackingserver eingesehen werden (http://localhost:5000)